In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!mkdir data
!cp '/content/drive/My Drive/AImongo_img.zip' .
!cp '/content/drive/My Drive/C1-P1_Test.zip' .

In [0]:
%%capture
!unzip AImongo_img.zip -d '/content/data/' 
!unzip C1-P1_Test.zip -d '/content/data/' 

In [4]:
!pip install pretrainedmodels
!pip install efficientnet_pytorch
!pip install torchtoolbox

     |████████████████████████████████| 61kB 1.8MB/s 
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-cp36-none-any.whl size=60962 sha256=aa7609a809e8408778ccfa67cd6d1b7cec42a8217bf83f76378f1e18829ea8aa
  Stored in directory: /root/.cache/pip/wheels/69/df/63/62583c096289713f22db605aa2334de5b591d59861a02c2ecd
Successfully built pretrainedmodels
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-cp36-none-any.whl size=12422 sha256=fd7b21bc08a9e1a50b39fff392e91050c84155c59078ab779b27b0d625f8f719
  Stored in directory: /root/.cache/pip/wheels/42/1e/a9/2a578ba9ad04e776e80bf0f70d8a7f4c29ec0718b92d8f6ccd
Successfully built efficientnet-pytorch
     |████████████████████████████████| 61kB 1.9MB/s 


In [0]:
import torch
import numpy as np
import pandas as pd
import joblib
import albumentations
import os
import glob
import cv2
from albumentations.core.transforms_interface import ImageOnlyTransform

# model.py
import torch.nn as nn
import pretrainedmodels
from torch.nn import functional as F
from efficientnet_pytorch import EfficientNet

# train
import pickle
# from tqdm import tqdm
# from tqdm import tqdm_notebook as tqdm
from tqdm.notebook import trange, tqdm
from torch.autograd import Variable
import argparse
from torch.utils.data import DataLoader
from torch.optim import Adam, AdamW
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from torchtoolbox.nn import LabelSmoothingLoss
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR
from itertools import chain

# Dataset

In [0]:
class ImageSamplerDataset:
    def __init__(self, phase, train_file, image_file_path, image_height, image_width, mean, std, binclass):
        self.image_file_path = image_file_path

        df = pd.read_csv(train_file)

        if binclass == 'A':
            class_map = {'A':1,'B':0,'C':0}

        elif binclass == 'B':
            class_map = {'A':0,'B':1,'C':0}

        elif binclass == 'C':
            class_map = {'A':0,'B':0,'C':1}

        else:
            class_map = {'A':0,'B':1,'C':2}

        self.img_id = df['image_id'].apply(lambda x: x.split('.')[0]).values # just take id of image_id
        self.labels = df['label'].apply(lambda x: x[-1]).map(class_map).values # encoding labels

        if phase == 'valid':
            # validation set
            self.aug = albumentations.Compose([
                albumentations.Resize(image_height, image_width),
                albumentations.Normalize(mean, std),
                # albumentations.ToFloat()
                ])
        elif phase == 'train':
            # training set
            self.aug = albumentations.Compose([
                albumentations.Resize(image_height, image_width),
                albumentations.RandomRotate90(p=0.5),
                albumentations.Transpose(p=0.5),
                albumentations.Flip(p=0.5),
                albumentations.OneOf([
                    albumentations.CLAHE(clip_limit=2), albumentations.IAASharpen(), albumentations.IAAEmboss(), 
                    albumentations.RandomBrightness(), albumentations.RandomContrast(),
                    albumentations.JpegCompression(), albumentations.Blur(), albumentations.GaussNoise()], p=0.5), 
                # albumentations.HueSaturationValue(p=0.5), 
                albumentations.ShiftScaleRotate(shift_limit=0.15, scale_limit=0.15, rotate_limit=45, p=0.5),
                albumentations.Normalize(mean, std),
                # albumentations.ToFloat()
                ])

    def __len__(self):
        return len(self.img_id)

    def __getitem__(self, item):
        img_bgr = cv2.imread(f"{self.image_file_path}/{self.img_id[item]}.jpg")
        img_rgb = img_bgr[:, :, [2, 1, 0]]
        image = self.aug(image=np.array(img_rgb))['image']
        image = np.transpose(image, [2, 0, 1]).astype(float)  # for using torchvision model
        return {
            'image': torch.tensor(image, dtype=torch.float),
            'label': torch.tensor(self.labels[item], dtype=torch.long)
        }

class ImageTestDataset:
    def __init__(self, file_path, image_height, image_width, mean, std):
        self.image_files = glob.glob(os.path.join(file_path, '*.jpg'))
        self.image_ids = [os.path.basename(f).split('.')[0] for f in self.image_files]

        # validation set
        self.aug = albumentations.Compose([
            albumentations.Resize(image_height,image_width,always_apply=True),
            albumentations.Normalize(mean,std,always_apply=True),
        ])
    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, item):
        img_bgr = cv2.imread(self.image_files[item])
        img_rgb = img_bgr[:, :, [2, 1, 0]]
        img = self.aug(image=np.array(img_rgb))['image']
        img_float = np.transpose(img, [2, 0, 1]).astype(float) # for using torchvision model
        return {
            'image' : torch.tensor(img_float, dtype=torch.float),
            'image_id' : self.image_ids[item]
        }

# Model

In [0]:
class EfficientNet_B6(nn.Module):
    def __init__(self, pretrained, n_class):
        super(EfficientNet_B6, self).__init__()
        if pretrained is True:
            self.model = EfficientNet.from_pretrained('efficientnet-b6')
        else:
            self.model = EfficientNet.from_pretrained('efficientnet-b6')

        self.l0 = nn.Linear(2304,n_class)

        self.classifier = nn.Sequential(
            nn.Linear(2304, 512),
            nn.ReLU(True),
            nn.Dropout(0.5),

            nn.Linear(512, 256),
            nn.ReLU(True),
            nn.Dropout(0.5),

            nn.Linear(256, 128),
            nn.ReLU(True),
            nn.Dropout(0.5),

            nn.Linear(128, n_class),
        )

    def forward(self, x):
        batch_size, _, _, _ = x.shape
        x = self.model.extract_features(x)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size,-1)
        output = self.classifier(x)

        return output
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

# Hyper-parameter Setting

In [0]:
MODEL_MEAN = (0.485,0.456,0.406)
MODEL_STD = (0.229,0.224,0.225)

train_file = 'data/AImongo_img/all_data.csv'
image_file = 'data/AImongo_img/'
image_height = 224
image_width = 224
num_workers = 4
device = 'cuda'
base_model = 'EfficientNet_B6'
lr = 3e-5
weight_decay = 3e-3
epochs = 25
train_batch_size = 16
test_batch_size = 32
test_size = 0.2
random_state = 2020
save_dir = 'data/.'
beta = 1
cutmix_prob = 0
binclass = None
nclass = 3

debug = True

# Train Function

In [0]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2


def loss_fn(outputs, target):
    class_weights = torch.tensor([[1, 2, 1]]).type(torch.FloatTensor).cuda() # hardcode class weight here
    loss = nn.CrossEntropyLoss(weight=class_weights)(outputs, target)
    return loss

def focal_loss_fn(outputs, target):
    alpha = torch.tensor(1.0, dtype=torch.float64, device=torch.device('cuda'))
    gamma = torch.tensor(2.0, dtype=torch.float64, device=torch.device('cuda'))

    target = target.view(-1,1)
    logpt = nn.functional.log_softmax(outputs, dim=1)
    logpt = logpt.gather(1, target)
    logpt = logpt.view(-1)
    pt = Variable(logpt.data.exp())

    loss = -alpha * (1-pt)**gamma * logpt
    return loss.mean()

def train(dataset_size, dataloader, model, optimizer, device, loss_fn):
    model.train()
    losses = AverageMeter()

    for batch_ind, d in tqdm(enumerate(dataloader), total=dataset_size/dataloader.batch_size):
        image = d['image']
        label = d['label']
        image = image.to(device,dtype=torch.float)
        target = label.to(device, dtype=torch.long)

        r = np.random.rand(1)
        if beta > 0 and r < cutmix_prob:
            # generate mixed sample
            lam = np.clip(np.random.beta(beta, beta), 0.3, 0.7)
            rand_index = torch.randperm(image.size()[0]).cuda()
            target_a = target
            target_b = target[rand_index]
            bbx1, bby1, bbx2, bby2 = rand_bbox(image.size(), lam)
            image[:, :, bbx1:bbx2, bby1:bby2] = image[rand_index, :, bbx1:bbx2, bby1:bby2]
            # adjust lambda to exactly match pixel ratio
            lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (image.size()[-1] * image.size()[-2]))
            # compute output
            outputs = model(image)
            loss = loss_fn(outputs, target_a) * lam + loss_fn(outputs, target_b) * (1. - lam)
        else:
            # compute output
            outputs = model(image)
            loss = loss_fn(outputs, target)

        losses.update(loss.item(), image.size(0))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return losses.avg


def evaluate(dataset_size, dataloader, model, device,loss_fn, tag):
    model.eval()
    losses = AverageMeter()
    image_pred_list = []
    image_target_list = []
    with torch.no_grad():
        for batch_ind, d in tqdm(enumerate(dataloader),total=dataset_size/dataloader.batch_size):
            image = d['image']
            label = d['label']
            image = image.to(device,dtype=torch.float)
            target = label.to(device, dtype=torch.long)
            outputs = model(image)

            loss = loss_fn(outputs, target)
            losses.update(loss.item(), image.size(0))

            pred_label = torch.argmax(outputs, dim=1)
            image_pred_list.append(pred_label)
            image_target_list.append(target)

    # Evaludation Metrics
    pred = torch.cat(image_pred_list).cpu().numpy()
    tgt = torch.cat(image_target_list).cpu().numpy()
    
    if not binclass:
        cfm = np.round(confusion_matrix(y_true=tgt, y_pred=pred, labels=[0,1,2]), 3)
    else:
        cfm = np.round(confusion_matrix(y_true=tgt, y_pred=pred, labels=[0,1]), 3)
    
    accu = accuracy_score(y_true=tgt,y_pred=pred)
    if tag == 'train':
        print(f'Confusion Matrix of {tag}')
        print(cfm)
        print('General Accuracy score on Train: {:5.4f}'.format(accu))
        return final_loss/counter, accu
    elif tag == 'valid':
        print(f'Confusion Matrix of {tag}')
        print(cfm)
        print('General Accuracy score on Valid: {:5.4f}'.format(accu))
        return losses.avg, accu

class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def get_train_valid_indice(test_size=0.2, random_state=42):
    df_tr = pd.read_csv(train_file)
    
    indice = df_tr.index
    label = df_tr.label

    train_indice, valid_indice, train_y, valid_y= train_test_split(indice, label, test_size=test_size, random_state=42, stratify=label)

    return train_indice, valid_indice

def model_dispatcher(if_pretrain, base_model, nclass):
    if base_model == 'se_resnext101_32x4d':
        return SE_ResNext101_32x4d(pretrained=if_pretrain, n_class=nclass)

    elif base_model == 'vgg16':
        return VGG16(pretrained=if_pretrain, n_class=nclass)

    elif base_model == 'resnet34': 
        return ResNet34(pretrained=if_pretrain, n_class=nclass)
    
    elif base_model == 'se_resnext101_32x4d_sSE': 
        return se_resnext101_32x4d_sSE(pretrained=if_pretrain, n_class=nclass)

    elif base_model == 'EfficientNet_B6': 
        return EfficientNet_B6(pretrained=if_pretrain, n_class=nclass)

def adjust_learning_rate(optimizer, epoch, step):
    lr = lr * (0.1 ** (epoch // step))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

# Train

In [6]:
if device =='cuda':
  torch.backends.cudnn.benchmark = True #  should add to speed up the code when input array shape doesn't vary
  print('Using cudnn.benchmark.')

model = model_dispatcher(True, base_model, nclass)
model.to(device)

train_indices, val_indices = get_train_valid_indice(test_size=test_size, random_state=random_state)

train_size = len(train_indices)
valid_size = len(val_indices)

train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_dataset = ImageSamplerDataset(
    phase = 'train',
    train_file = train_file,
    image_file_path = image_file,
    image_height=image_height,
    image_width=image_width,
    mean=MODEL_MEAN,
    std=MODEL_STD,
    binclass = binclass
)

train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=train_batch_size,
    num_workers=num_workers,
    sampler=train_sampler
)

valid_dataset = ImageSamplerDataset(
    phase = 'valid',
    train_file = train_file,
    image_file_path=image_file,
    image_height=image_height,
    image_width=image_width,
    mean=MODEL_MEAN,
    std=MODEL_STD,
    binclass = binclass
)

valid_dataloader = DataLoader(
    dataset=valid_dataset,
    batch_size=test_batch_size,
    num_workers=num_workers,
    sampler=valid_sampler
)
optimizer = AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
# scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=5, factor=0.3)
scheduler_cosine = CosineAnnealingLR(optimizer, epochs)

if torch.cuda.device_count() > 1 :
    model = nn.DataParallel()

val_accu_benchmark = 0.34
best_epoch = 0

for epoch in range(epochs):
    tr_loss = train(dataset_size=train_size ,dataloader=train_dataloader, model=model, optimizer=optimizer, device=device, loss_fn=focal_loss_fn)
    print(f'Epoch_{epoch+1} Train Loss:{tr_loss}')

    if debug:
        val_loss, val_accu = evaluate(dataset_size=valid_size, dataloader=valid_dataloader, model=model, device=device, loss_fn=focal_loss_fn, tag='valid')
        print(f'Epoch_{epoch+1} Valid Loss:{val_loss}')
        # scheduler.step(val_loss)
        
        if val_accu > val_accu_benchmark:
            best_epoch = epoch+1
            print(f'save {base_model} model on epoch {epoch+1}')
            torch.save(model.state_dict(), os.path.join(save_dir, f'{base_model}.bin'))
            val_accu_benchmark = val_accu
    else:
        if epoch == epochs - 1:
            torch.save(model.state_dict(), os.path.join(save_dir, f'{base_model}.bin'))

    scheduler_cosine.step(epoch)

print(f'Save the best model on epoch {best_epoch}')

Using cudnn.benchmark.
Loaded pretrained weights for efficientnet-b6



Epoch_1 Train Loss:0.48396225897595285



Confusion Matrix of valid
[[ 85 312  10]
 [ 10 368  94]
 [  2  84 315]]
General Accuracy score on Valid: 0.6000
Epoch_1 Valid Loss:0.4600554287433624
save EfficientNet_B6 model on epoch 1


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:143: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)



Epoch_2 Train Loss:0.3451188722159714


...


Confusion Matrix of valid
[[355  50   2]
 [ 98 316  58]
 [  4  37 360]]
General Accuracy score on Valid: 0.8055
Epoch_23 Valid Loss:0.14780369009822608



Epoch_24 Train Loss:0.12500444682082162



Confusion Matrix of valid
[[357  48   2]
 [103 310  59]
 [  4  35 362]]
General Accuracy score on Valid: 0.8039
Epoch_24 Valid Loss:0.14793346244841815



Epoch_25 Train Loss:0.12563720986945554



Confusion Matrix of valid
[[356  49   2]
 [ 99 314  59]
 [  3  35 363]]
General Accuracy score on Valid: 0.8070
Epoch_25 Valid Loss:0.14724463745951652
Save the best model on epoch 11


# Train on All

In [0]:
MODEL_MEAN = (0.485,0.456,0.406)
MODEL_STD = (0.229,0.224,0.225)

train_file = 'data/AImongo_img/all_data.csv'
image_file = 'data/AImongo_img/'
image_height = 224
image_width = 224
num_workers = 4
device = 'cuda'
base_model = 'EfficientNet_B6'
lr = 3e-5
weight_decay = 3e-3
epochs = 11
train_batch_size = 16
test_batch_size = 32
random_state = 2020
save_dir = 'data/.'
beta = 1
cutmix_prob = 0
binclass = None
nclass = 3

In [12]:
if device =='cuda':
    torch.backends.cudnn.benchmark = True #  should add to speed up the code when input array shape doesn't vary
    print('Using cudnn.benchmark.')

model = model_dispatcher(True, base_model, nclass)
model.to(device)

train_size = len(pd.read_csv(train_file))
print(train_size)

train_dataset = ImageSamplerDataset(
    phase = 'train',
    train_file = train_file,
    image_file_path = image_file,
    image_height=image_height,
    image_width=image_width,
    mean=MODEL_MEAN,
    std=MODEL_STD,
    binclass = binclass
)

train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=train_batch_size,
    num_workers=num_workers
)

optimizer = AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler_cosine = CosineAnnealingLR(optimizer, epochs)

if torch.cuda.device_count() > 1 :
    model = nn.DataParallel()


for epoch in range(epochs):
    tr_loss = train(dataset_size=train_size ,dataloader=train_dataloader, model=model, optimizer=optimizer, device=device, loss_fn=focal_loss_fn)
    print(f'Epoch_{epoch+1} Train Loss:{tr_loss}')

    scheduler_cosine.step(epoch)

torch.save(model.state_dict(), os.path.join(save_dir, f'{base_model}_on_all_epoch11.bin'))
print('train on all is complete')


Using cudnn.benchmark.


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b6-c76e70fd.pth" to /root/.cache/torch/checkpoints/efficientnet-b6-c76e70fd.pth



Loaded pretrained weights for efficientnet-b6
6400



Epoch_1 Train Loss:0.4532538520544767


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:143: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


...


Epoch_7 Train Loss:0.1728130916133523



Epoch_8 Train Loss:0.16624490132555367



Epoch_9 Train Loss:0.16166081985458733



Epoch_10 Train Loss:0.15749956410378219



Epoch_11 Train Loss:0.15454106766730547
train on all is complete


In [13]:
from google.colab import files
files.download('data/EfficientNet_B6_on_all_epoch11.bin') 

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 45330, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil

# Inference

In [0]:
class ImageTTADataset:
    def __init__(self, file_path, transform):
        self.image_files = glob.glob(os.path.join(file_path, '*.jpg'))
        self.image_ids = [os.path.basename(f).split('.')[0] for f in self.image_files]

        # validation set
        self.aug = transform

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, item):
        img_bgr = cv2.imread(self.image_files[item])
        img_rgb = img_bgr[:, :, [2, 1, 0]]
        img = self.aug(image=np.array(img_rgb))['image']
        img_float = np.transpose(img, [2, 0, 1]).astype(float) # for using torchvision model
        return {
            'image' : torch.tensor(img_float, dtype=torch.float),
            'image_id' : self.image_ids[item]
        }

In [0]:
%%capture

!cp '/content/drive/My Drive/best_efficientnet_b6.zip' .

!unzip best_efficientnet_b6.zip -d '/content/data/' 

In [0]:
n_tta = 40
output_name = 'tta_submission_all_11epochs'
save_dir = 'data'
model_weights = 'EfficientNet_B6_on_all_epoch11.bin'
image_file = 'data/C1-P1_Test'

In [16]:
data_transforms = albumentations.Compose([
    albumentations.Resize(image_height, image_width),
    albumentations.RandomRotate90(p=0.5),
    albumentations.Transpose(p=0.5),
    albumentations.Flip(p=0.5),
    albumentations.OneOf([
        albumentations.CLAHE(clip_limit=2), albumentations.IAASharpen(), albumentations.IAAEmboss(), 
        albumentations.RandomBrightness(), albumentations.RandomContrast(),
        albumentations.JpegCompression(), albumentations.Blur(), albumentations.GaussNoise()], p=0.6), 
    # albumentations.HueSaturationValue(p=0.5), 
    albumentations.ShiftScaleRotate(shift_limit=0.3, scale_limit=0.3, rotate_limit=45, p=0.9),
    albumentations.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
    # albumentations.ToFloat()
    ])

data_transforms_test = albumentations.Compose([
    albumentations.Resize(image_height, image_width),
    albumentations.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
    # albumentations.ToFloat()
    ])

data_transforms_tta0 = albumentations.Compose([
    albumentations.Resize(image_height, image_width),
    albumentations.RandomRotate90(p=0.5),
    albumentations.Transpose(p=0.5),
    albumentations.Flip(p=0.5),
    albumentations.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
    # albumentations.ToFloat()
    ])

data_transforms_tta1 = albumentations.Compose([
    albumentations.Resize(image_height, image_width),
    albumentations.RandomRotate90(p=1),
    albumentations.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
    # albumentations.ToFloat()
    ])

data_transforms_tta2 = albumentations.Compose([
    albumentations.Resize(image_height, image_width),
    albumentations.Transpose(p=1),
    albumentations.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
    # albumentations.ToFloat()
    ])

data_transforms_tta3 = albumentations.Compose([
    albumentations.Resize(image_height, image_width),
    albumentations.Flip(p=1),
    albumentations.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
    # albumentations.ToFloat()
    ])

model = model_dispatcher(False, base_model, nclass)
model.to(device)
model.load_state_dict(torch.load(os.path.join(save_dir, model_weights)))

model.eval()
print(f'Loading pretrained model: {base_model} for eval')

for num_tta in range(n_tta):
    if num_tta == 0:
        test_dataset = ImageTTADataset(file_path = image_file, transform=data_transforms_test)
        test_dataloader =DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False, num_workers=num_workers)

    elif num_tta == 1:
        test_dataset = ImageTTADataset(file_path = image_file, transform=data_transforms_tta1)
        test_dataloader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False, num_workers=num_workers)

    elif num_tta == 2:
        test_dataset = ImageTTADataset(file_path = image_file, transform=data_transforms_tta2)
        test_dataloader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False, num_workers=num_workers)

    elif num_tta == 3:
        test_dataset = ImageTTADataset(file_path = image_file, transform=data_transforms_tta3)
        test_dataloader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False, num_workers=num_workers)

    elif num_tta < 8:
        test_dataset = ImageTTADataset(file_path = image_file, transform=data_transforms_tta0)
        test_dataloader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False, num_workers=num_workers)

    else:
        test_dataset = ImageTTADataset(file_path = image_file, transform=data_transforms)
        test_dataloader =DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False, num_workers=num_workers)


    image_id_list = []
    image_pred_list = []

    with torch.no_grad():
        for d in test_dataloader:
            image = d['image']
            img_id = d['image_id']

            image = image.to(device, dtype=torch.float)
            outputs = model(image)
            pred_prob = torch.nn.Softmax(dim=1)(outputs)

            image_id_list.append(img_id)
            image_pred_list.append(pred_prob/n_tta)
        
        if num_tta == 0:
            ids = list(chain(*image_id_list))
            preds = torch.cat(image_pred_list).cpu().numpy()

        else:
            preds_tmp = torch.cat(image_pred_list).cpu().numpy()
            preds += preds_tmp

    print(num_tta)

df_pred = pd.DataFrame(preds, columns=['A', 'B', 'C'])
df_id = pd.DataFrame(ids, columns=['image_ids'])
sub = pd.concat([df_id, df_pred], axis=1)
sub.to_csv(f'{output_name}.csv',index=False)

Loaded pretrained weights for efficientnet-b6
Loading pretrained model: EfficientNet_B6 for eval
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


# Check Submission

In [0]:
sub = sub.sort_values('image_ids')
sub.to_csv('tta_submission_all_11epochs.csv', index=False)

In [22]:
sub

,image_ids,A,B,C
5,00001,0.011332,0.240750,0.747918
1123,00006,0.676293,0.310405,0.013301
1254,00011,0.290767,0.680404,0.028829
1185,00016,0.655755,0.328967,0.015278
1430,00025,0.004312,0.083193,0.912496
...,...,...,...,...
798,07977,0.596139,0.378463,0.025399
408,07991,0.012411,0.188638,0.798951
575,07992,0.676379,0.304764,0.018857
90,07994,0.459947,0.498477,0.041576


In [24]:
sub[['A', 'B', 'C']].apply(lambda x: np.argmax(x, axis=0), axis=1).value_counts()

0    701
2    487
1    412
dtype: int64

In [0]:
sub['label'] = sub[['A', 'B', 'C']].apply(lambda x: np.argmax(x, axis=0), axis=1)
sub = sub.drop(['A','B','C'], axis=1)

class_map ={0:'A',1:'B',2:'C'}
sub['label'] = sub['label'].map(class_map)